In [ ]:
# Puzzle 1
def get_data(filename):
    l_bricks = []
    max_x = 0
    max_y = 0
    max_z = 0
    
    with open(filename, "r") as file:
        for line in file:
            line = line.strip()
            b1, b2 = line.split("~")
            b1 = b1.split(",")
            b2 = b2.split(",")
            b1 = [int(x) for x in b1]
            b2 = [int(x) for x in b2]
            x1, y1, z1 = b1
            x2, y2, z2 = b2
            if max(x1, x2) > max_x:
                max_x = max(x1, x2)
            if max(y1, y2) > max_y:
                max_y = max(y1, y2)
            if max(z1, z2) > max_z:
                max_z = max(z1, z2)
            l_bricks.append(((x1, y1, z1), (x2, y2, z2)))
    return l_bricks, max_x, max_y, max_z
    
def check_if_brick_is_long(brick):
    b1, b2 = brick
    x1, y1, z1 = b1
    x2, y2, z2 = b2
    x = x2 - x1
    y = y2 - y1
    z = z2 - z1
    if (x < 0) or (y < 0) or (z < 0):
        print("Verdreht", brick)
    if not((x == 0 and y == 0) or (x == 0 and z == 0) or (y == 0 and z == 0)):
        print("Mehrdimensional",  brick)
        
def init_z_plan(max_x, max_y):
    z_plan = []
    for x in range(max_x + 1):
        row = []
        for y in range(max_y + 1):
            row.append(0)
        z_plan.append(row)
    return z_plan

def print_z_plan(z_plan):
    for row in z_plan:
        for col in row:
            print(f"{col:4d}", end = "")
        print()
    print()
    
def sort_brick_by_z(l_bricks):
    l_sort_bricks = []
    while len(l_bricks) > 0:
        min_z = l_bricks[0][0][2]
        min_brick = l_bricks[0]
        for brick in l_bricks:
            if brick[0][2] < min_z:
                min_z = brick[0][2]
                min_brick = brick
        l_sort_bricks.append(min_brick)
        l_bricks.remove(min_brick)
    return l_sort_bricks
    
def drop_brick(z_plan, brick):
    block1, block2 = brick
    x1, y1, z1 = block1
    x2, y2, z2 = block2
    min_z = 0
    for x in range(x1, x2 + 1):
        for y in range(y1, y2 + 1):
            if z_plan[x][y] > min_z:
                min_z = z_plan[x][y]
    min_z += 1
    for x in range(x1, x2 + 1):
        for y in range(y1, y2 + 1):
            z_plan[x][y] = min_z + z2 - z1
    block1 = (x1, y1, min_z)
    block2 = (x2, y2, min_z + z2 - z1)
    brick = (block1, block2)
    return z_plan, brick

def touches(brick1, brick2):
    block1, block2 = brick1
    x1, y1, z1 = block1
    x2, y2, z2 = block2
    block3, block4 = brick2
    x3, y3, z3 = block3
    x4, y4, z4 = block4
    for x in range(x1, x2 + 1):
        for y in range(y1, y2 + 1):
            if (x3 <= x <= x4) and (y3 <= y <= y4):
                return True
    return False    
                            
def create_support_lists(l_bricks):
    supports_brick = {}
    supported_by_brick = {}
    for brick in l_bricks:
        supports_brick[brick] = []
        z = brick[1][2]
        for supported_brick in l_bricks:
            if supported_brick[0][2] == z + 1:
                if touches(brick, supported_brick):
                    supports_brick[brick].append(supported_brick)
                    if supported_brick in supported_by_brick:
                        supported_by_brick[supported_brick].append(brick)
                    else:
                        supported_by_brick[supported_brick] = [brick]
    return supports_brick, supported_by_brick

def disintegrate_brick(l_bricks, dis_brick, supports_brick, supported_by_brick):
    l_fall = [dis_brick]
    l_potential_fall = []
    for brick in supports_brick[dis_brick]:
        l_potential_fall.append(brick)
    while l_potential_fall != []:
        pot_fall_brick = l_potential_fall[0]
        l_potential_fall.remove(pot_fall_brick)
        for brick in supports_brick[pot_fall_brick]:
            if not(brick in l_potential_fall):
                l_potential_fall.append(brick)
        fall = True
        for brick in supported_by_brick[pot_fall_brick]:
            if not(brick in l_fall):
                fall = False
        if fall:
            l_fall.append(pot_fall_brick)
    
    return len(l_fall) - 1
    
          
# Main Program
#Get Data
l_bricks, max_x, max_y, max_z = get_data("Puzzle.txt")   
# Initialize plan, where the current z-value is given for each xy-block
z_plan = init_z_plan(max_x, max_y)
# Sort all the bricks by the lower z-value
l_bricks = sort_brick_by_z(l_bricks)
# Let all brick drop down, create a list of dropped bricks
l_dropped_bricks = []
for brick in l_bricks:
    z_plan, dropped_brick = drop_brick(z_plan, brick)
    l_dropped_bricks.append(dropped_brick)

# Go throught the list and check, which brick support which brick and vice versa
supports_brick, supported_by_brick = create_support_lists(l_dropped_bricks)


# Count those brick, which do NOT support another brick or do not ALONE support a brick
can_be_deleted = 0
deleted_bricks = []
for brick in l_dropped_bricks:
    if supports_brick[brick] == []:
        can_be_deleted += 1
        deleted_bricks.append(brick)
    else:
        could_be_deleted = True
        for supported_brick in supports_brick[brick]:
            if len(supported_by_brick[supported_brick]) == 1:
                could_be_deleted = False
        if could_be_deleted:
            can_be_deleted += 1
            deleted_bricks.append((brick))
            
print(can_be_deleted, "bricks can be deleted")            

# For each brick, count the number of fallen bricks, if it was disintegrated

num_fall_bricks = 0

for brick in l_dropped_bricks:
    num_fall_bricks += disintegrate_brick(l_dropped_bricks, brick, supports_brick, supported_by_brick)

print(num_fall_bricks, "will fall")